In [1]:
import pandas as pd
from prophet import Prophet
import numpy as np

In [2]:
df_train = pd.read_parquet('AQI_Historical_New')

In [3]:
df_train = df_train[df_train['TRT_ID'] == 20]

In [4]:
df_train['lat,lon,trt'] = 'Lat:' + df_train['coord.lat'].astype(str) + ', Lon:'+ df_train['coord.lon'].astype(str) + ', TRT_ID:' + df_train['TRT_ID'].astype(str)

In [5]:
df_train = df_train.drop('coord.lon', axis=1)
df_train = df_train.drop('coord.lat', axis=1)
df_train = df_train.drop('TRT_ID', axis=1)
#df_train = df_train.drop('main.aqi', axis=1)

In [6]:
df_train["dt"] = pd.to_datetime(df_train["dt"])

In [7]:
df_train = df_train.rename(columns={'dt':'ds','components.pm2_5':'y'})

In [8]:
df_train['y_orig'] = df_train['y']
df_train['y'] = np.log(df_train['y'])

In [9]:
model_new = Prophet()
model_new.add_regressor('components.co')
model_new.add_regressor('components.no')
model_new.add_regressor('components.no2')
model_new.add_regressor('components.o3')
model_new.add_regressor('components.so2')
model_new.add_regressor('components.pm10')
model_new.add_regressor('components.nh3')

In [10]:
model_new.fit(df_train)

12:00:03 - cmdstanpy - INFO - Chain [1] start processing
12:00:12 - cmdstanpy - INFO - Chain [1] done processing


In [11]:
future_data = model_new.make_future_dataframe(periods=720, freq='H')

In [ ]:
future_data = df[['ds','components.co', 'components.no', 'components.no2', 'components.o3', 'components.so2', 'components.pm10', 'components.nh3']]

In [6]:
df_train = df_train.set_index('dt').groupby('lat,lon,trt').resample('D').mean()

In [7]:
df_train = df_train.reset_index()

In [10]:
locations = df_train.groupby("lat,lon,trt")

In [17]:
for location in locations.groups:
    group = locations.get_group(location)
    train_set = group[(group['ds'] >= '2021-01-01') & (group['ds'] <= '2022-11-30')]
    test_set = group[(group['ds'] > '2022-11-30')]

In [11]:
target = pd.DataFrame()

In [15]:
for location in locations.groups:
    group = locations.get_group(location)

    model_new = Prophet(interval_width=0.95)
    model_new.add_regressor('components.co')
    model_new.add_regressor('components.no')
    model_new.add_regressor('components.no2')
    model_new.add_regressor('components.o3')
    model_new.add_regressor('components.so2')
    model_new.add_regressor('components.pm10')
    model_new.add_regressor('components.nh3')

    model_new.fit(group)
    future = model_new.make_future_dataframe(periods=365)
    forecast = model_new.predict(future)
    forecast = forecast.rename(columns={'yhat':'yhat_' + location})
    target = pd.merge(target, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)


10:55:06 - cmdstanpy - INFO - Chain [1] start processing
10:55:06 - cmdstanpy - INFO - Chain [1] done processing


ValueError: Regressor 'components.co' missing from dataframe

In [8]:
df_train = df_train.sort_values('ds')

In [10]:
model_new = Prophet(interval_width=0.95, yearly_seasonality=True)
model_new.add_regressor('components.co')
model_new.add_regressor('components.no')
model_new.add_regressor('components.no2')
model_new.add_regressor('components.o3')
model_new.add_regressor('components.so2')
model_new.add_regressor('components.pm10')
model_new.add_regressor('components.nh3')

In [11]:
model_new.fit(df_train)

13:18:51 - cmdstanpy - INFO - Chain [1] start processing
13:19:00 - cmdstanpy - INFO - Chain [1] done processing


In [12]:
future = model_new.make_future_dataframe(periods=700)
future.tail()

,ds
17999,2024-11-26 23:00:00
18000,2024-11-27 23:00:00
18001,2024-11-28 23:00:00
18002,2024-11-29 23:00:00
18003,2024-11-30 23:00:00


In [13]:
future = pd.concat([future, df_train], axis=1)